<a href="https://colab.research.google.com/github/Rubnserrano/DataScience/blob/main/framework_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
D_in, H, D_out = 2, 3, 2

mlp = MLP([
    Linear(D_in, H),
    Relu(),
    Linear(H, D_out)
])

optimizer = SGD(mlp, lr=0.01)
loss = CrossEntropy(mlp)

epochs = 100
for e in range(epochs):
  for batch in batches:
    y_pred = mlp(x)
    loss(y_pred, y)
    loss.backward() ## Calcular gradientes
    optimizer.update()

In [ ]:
class MLP:
  def __init__(self, layers):
    # lista de capas
    self.layers = layers
  
  def __call__(self, x):

    for layer in self.layers:
      x = layer(x)
    return x

In [ ]:
class Layer():
  def __init__(self):
    self.params = []
    self.grads = []

  def __call__(self,x):
    return x

  def backward(self, grad):
    return grad

  def update(self, params):
    return

In [ ]:
class Linear(Layer):
  def __init__(self, d_in, d_out):
    self.w = np.random.normal(loc=0.0,
                              scale = np.sqrt(2/(d_in+d_out)),
                              size = (d_in, d_out))
    self.b = np.zeros(d_out)

  def __call__(self, x):
    self.x = x
    self.params = [self.w, self.b]
    return np.dot(x, self.w) + self.b
  
  def backward(self, grad_output):
    grad = np.dot(grad_output, self.w.T)
    self.grad_w = np.dot(self.x.T, grad_output)

    self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
    self.grads = [self.grad_w, self.grad_b]
    return grad

  def update(self, params):
    self.w, self.b = params

In [ ]:
class ReLU(layer):
  def __call__(self, x):
    self.x = x
    return np.maxium(0, x)

  def backward(self, grad_output):
    grad = self.x > 0
    return grad_output*grad
  
def sigmoid(x):
  return 1 / (1+np.exp(-x))

def softmax(x):
  return np.exp(x) / np.exp(x).sum(axis=-1, keepdims = True)

class Sigmoid(Layer):
  def __call__(self, x):
    self.x = x
    return sigmoid(x)

  def backward(self, grad_output):
    grad = sigmoid(self.x)*(1-sigmoid(self.x))
    return grad_output*grad

In [ ]:
class SGD():
  def __init__(self, net, lr):
    self.net = net
    self.lr = lr

  def update(self):
    for layer in self.net.layers:
      layer.update([
          params - self.lr*grads
          for params, grads in zip(layer.params, layer.grads)
      ])

In [ ]:
class Loss():
  def __init__(self, net):
    self.net = net

  def backward(self):
    grad = self.grad_loss()

  for layer in reversed(self.net.layers):
    grad = layer.backward(grad)


class MSE(Loss):
  def __call__(self, output, target):
    self.output, self.target = output, target.reshape(output.shape)
    loss = np.mean((self.output - self.target)**2)
    return loss.mean()

    def grad_loss(self):
      return self.output - self.target

  
class BCE(Loss):
  def __call__(self, output, target):
    self.output, self.target = output, target.reshape(output.shape)
    loss = - np.mean(self.target*np.log(self.output) - (1-self.target)*np.log(1-self.output))
    return loss.mean()

    def grad_loss(self):
      return self.output - self.target


class CrossEntropy(Loss):
  def __call__(self, output, target):
    self.output, self.target = output, target
    logits = output[np.arange(len(output)), target]
    loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
    loss = loss.mean()
    return layer

  def grad_loss(self):
    answers = np.zeros_like(self.output)
    answers[np.arnge(len(self.output)), self.target] = 1
    return (-answers + softmax(self.output)) / self.output.shape[0]
    